# Report of Project 4 

## Visualizing Sliding Window and Flow Control for TCP Via Python

### Author: Qiong Yang

## 1 Principles of Reliable Data Transfer

### 1.1 Go-Back-N

In a Go-Back-N (GBN) protocol, the sender is allowed to transmit multiple packets
(when available) without waiting for an acknowledgment, but is constrained to have no
more than some maximum allowable number, N, of unacknowledged packets in the
pipeline. 

In GBN, we define  textbf{base} to be the sequence number of the oldest unacknowledged
packet and textbf{nextseqnum} to be the smallest unused sequence number (that is, the
sequence number of the next packet to be sent)
We can devide the range of sequence numbers into four intervals:[0,base-1],[base,nextseqnum-1],[nextsequem,base+N-1] and [base+N,default].

-  interval [0,base-1]:correspond to packets that have already been transmitted and acknowledged

- interval [base,nextseqnum-1]:corresponds to packets that have been sent but not yet
acknowledged

- interval [nextsequem,base+N-1]:be used for packets that can be sent immediately

- interval [base+N,default]:cannot be used until
	an unacknowledged packet currently in the pipeline (specifically, the packet with
	sequence number base) has been acknowledged.

### 1.2 Simulation 

In [1]:
import wx


class MyFrame(wx.Frame):
    global_count = 0
    def __init__(self, parent, title):
        wx.Frame.__init__(self, parent, title = title, size = (800, 500))
        self.initFrame()


    def initFrame(self):
        self.Mylayout()
        self.Mycreatepacket()
        
        self.Centre()
        self.Show(True)

        # bind events
        self.timer = wx.Timer(self)
        self.Bind(wx.EVT_TIMER, self.TimerHandler)

        #self.timer.Start(10)
        self.Bind(wx.EVT_BUTTON, self.OnClick, self.my_button)
        self.Bind(wx.EVT_PAINT, self.Mydraw)
        self.timer1=1
        self.tmp=0

    def Mylayout(self):
        self.vbox = wx.BoxSizer(wx.VERTICAL)
        self.my_button = wx.Button(self, label = 'start')
        self.vbox.Add(self.my_button, flag = wx.ALL, border = 4)
        self.SetSizer(self.vbox)
        self.my_static_text = wx.StaticText(self, label = "input time", pos = (135, 10))
        self.my_static_text = wx.StaticText(self, label = "count down", pos = (470, 10))
        self.my_spinctrl = wx.SpinCtrl(self, value = '0', pos = (550, 10))
        self.my_spinctrl.SetRange(0, 20)
        self.my_spinctrl2 = wx.SpinCtrl(self, value = '0', pos = (200, 10))
        self.my_spinctrl2.SetRange(0, 20)
        #self.rtt=self.my_spinctrl2.GetValue()


    def OnClick(self, event):
        self.timer.Start(40)
        self.rtt = self.my_spinctrl2.GetValue()
        self.timer1=int(self.rtt)
        # pass

    def TimerHandler(self, event):
        MyFrame.global_count = MyFrame.global_count + 1
        self.tmp=self.tmp+1
        if self.tmp>5:
            self.tmp=0
        if self.timer1>0:
            self.timer1=self.timer1-self.tmp/5
        else:
            self.timer1=self.rtt
        self.my_spinctrl.SetValue(self.timer1)
        self.send()
        for i in range(len(self.flying_packet1)):
            if self.flying_packet1[i].direction == "down" and self.flying_packet1[i].y == 50:
                self.flying_packet1[i].start_time =MyFrame.global_count

            if MyFrame.global_count - self.flying_packet1[i].start_time > self.rtt:
                self.send()
        self.Refresh()

    def Mycreatepacket(self):
        self.packet_num = 15
        self.winsize=5
        self.top_packet = [MyPacket("top","blue","null",i*50+20, 50) for i in range(self.packet_num)]
        self.bottom_packet = [MyPacket("bottom", "white","null", i*50+20, 350) for i in range(self.packet_num)]
        self.flying_packet1 = [MyPacket("middle", "blue","down", i*50+20, 50-i*30) for i in range(self.winsize)]
        self.top1_packet = [MyPacket("top","yellow","null",k*50+20,50) for k in range(5)]
        self.bottom1_packet = [MyPacket("bottom", "dark blue", "null", k*50+20, 350) for k in range(5)]
        self.slider_window = SliderWindow(15,45)
    
    def Mydraw(self, event):
        dc = wx.PaintDC(self)

        dc.SetPen(wx.Pen('#4c4c4c', 1, wx.SOLID))
        dc.DrawRectangle(self.slider_window.x, self.slider_window.y, 250, 50)
        
        dc.SetBrush(wx.Brush("#F9F900", wx.SOLID))
        for i in range(5):
            dc.SetPen(wx.Pen('#4c4c4c', 1, wx.SOLID))
            dc.DrawRectangle(self.top1_packet[i].x, self.top1_packet[i].y, 20, 40)

        dc.SetBrush(wx.Brush("#000080", wx.SOLID))
        for i in range(5):
            dc.SetPen(wx.Pen('#4c4c4c', 1, wx.SOLID))
            dc.DrawRectangle(self.bottom1_packet[i].x, self.bottom1_packet[i].y, 20, 40)


        for i in range(self.packet_num):
            if self.top_packet[i].color == "blue":
                dc.SetBrush(wx.Brush("#46A3FF", wx.SOLID))
                dc.SetPen(wx.Pen('#4c4c4c', 1, wx.SOLID))
                dc.DrawRectangle(self.top_packet[i].x, self.top_packet[i].y, 20, 40)

            if self.top_packet[i].color == "yellow":
                dc.SetBrush(wx.Brush("#F9F900", wx.SOLID))
                dc.SetPen(wx.Pen('#4c4c4c', 1, wx.SOLID))
                dc.DrawRectangle(self.top_packet[i].x, self.top_packet[i].y, 20, 40)
        
        for i in range(self.packet_num):
            if self.bottom_packet[i].color == "white":
                dc.SetBrush(wx.Brush("#FCFCFC", wx.SOLID)) 
                dc.SetPen(wx.Pen('#4c4c4c', 1, wx.SOLID))
                dc.DrawRectangle(self.bottom_packet[i].x, self.bottom_packet[i].y, 20, 40)
            if self.bottom_packet[i].color ==  "dark blue":
                dc.SetBrush(wx.Brush("#000080", wx.SOLID))
                dc.SetPen(wx.Pen('#4c4c4c', 1, wx.SOLID))
                dc.DrawRectangle(self.bottom_packet[i].x, self.bottom_packet[i].y, 20, 40)
        

        for i in range(self.winsize):
            if self.flying_packet1[i].y>=50:
                if self.flying_packet1[i].color == "blue":
                    dc.SetBrush(wx.Brush("#46A3FF", wx.SOLID))
                    dc.SetPen(wx.Pen('#4c4c4c', 1, wx.SOLID))
                    dc.DrawRectangle(self.flying_packet1[i].x, self.flying_packet1[i].y, 20, 40)
                if self.flying_packet1[i].color == "green":
                    dc.SetBrush(wx.Brush("#8cEA00", wx.SOLID))
                    dc.SetPen(wx.Pen('#4c4c4c', 1, wx.SOLID))
                    dc.DrawRectangle(self.flying_packet1[i].x, self.flying_packet1[i].y, 20, 40)
 
    def send(self):
         
        for i in range(len(self.flying_packet1)):
            
            if self.flying_packet1[i].direction == "down":
                self.flying_packet1[i].y = self.flying_packet1[i].y +1
                
                if self.flying_packet1[i].y == 350:
                    #print k 
                    self.flying_packet1[i].color = "green"
                    self.bottom_packet[i].color = "dark blue"
                    
                    self.flying_packet1[i].direction = "up"
            if self.flying_packet1[i].direction == "up":
                
                self.flying_packet1[i].y  = self.flying_packet1[i].y - 1
                if self.flying_packet1[i].y == 50:
                                self.timer1=self.rtt
                                self.tmp=0
                                self.my_spinctrl.SetValue(self.rtt)
                                self.top_packet[i].color = "yellow"                                
                                self.slider_window.x = self.slider_window.x + 50
                                self.flying_packet1[i].x = self.flying_packet1[i].x + 250
                                self.flying_packet1[i].y = 50
                                self.flying_packet1[i].direction = "down"
                                self.flying_packet1[i].color = "blue"

                                self.top_packet[i] = self.top_packet[i+5] 
                                self.bottom_packet[i] = self.bottom_packet[i+5]
                                if self.flying_packet1[i].x >= 520:
                                    self.top_packet[i] = self.top_packet[i+10]
                                    self.bottom_packet[i] = self.bottom_packet[i+10]
                                
                                if self.slider_window.x >= 765 and self.flying_packet1[i].x >= 770:
                                    self.flying_packet1 = [MyPacket("middle", "blue","down", i*50+20, 50-i*30) for i in range(self.winsize)]
                                    self.slider_window.x = 15
                                    self.slider_window.y = 45
                                    self.top_packet = [MyPacket("top","blue","null",i*50+20, 50) for i in range(self.packet_num)]
                                    self.bottom_packet = [MyPacket("bottom", "white","null", i*50+20, 350) for i in range(self.packet_num)]
    
class MyPacket(object):
    def __init__(self, state, color, direction,x,y ):
        self.state = state
        self.color = color
        self.direction = direction
        self.x = x
        self.y = y
        self.timer =20
        self.start_time = 0

class SliderWindow(object):
    def __init__(self,x,y):
        self.x = x
        self.y = y
app = wx.App()
MyFrame(None, title = "GO Back N")
app.MainLoop()


## 2 Flow Control

We know that the hosts on each side of a TCP connection set aside a receive buffer for
the connection. When the TCP connection receives bytes that are correct and in
sequence, it places the data in the receive buffer. The associated application process
will read data from this buffer, but not necessarily at the instant the data arrives.

Indeed, the receiving application may be busy with some other task and may not
even attempt to read the data until long after it has arrived. If the application is relatively slow at reading the data, the sender can very easily overflow the connection’s
receive buffer by sending too much data too quickly.

TCP provides a flow-control service to its applications to eliminate the possibility
of the sender overflowing the receiver’s buffer. Flow control is thus a speed-matching
service—matching the rate at which the sender is sending against the rate at which the
receiving application is reading. As noted earlier, a TCP sender can also be throttled
due to congestion within the IP network; this form of sender control is referred to as
congestion control.

### 2.1 Simulation

For convenience of show the change of buffer , I assume that there are three buffers  and applications in sender and receiver. However, in reality, three buffers respectively are the same.  

In [1]:
import wx

class MyFrame(wx.Frame):
    def __init__(self, parent, title):
        wx.Frame.__init__(self, parent, title = title, size = (800, 400))
        self.initFrame()

    def initFrame(self):
        self.buffer_size = 2048
        self.file_size = 4096
        self.file_block = 2048
        self.count1 =0
        self.count2 =0
        self.count3 =2048
        self.count4 = 0
        self.count5 =0
        self.count6 = 2048
        self.count7 = 2048
        self.count8 =2048
        self.count9 = 2048
        self.count10 = 0
        #self.flag = 0
        self.Mylayout()
        self.Centre(True)
        self.Bind(wx.EVT_IDLE, self.OnIdle)
        self.Show(True)

    def Mylayout(self):
        
        self.my_static_text1 = wx.StaticText(self, label = "sender buffer", pos = (10, 10))
        self.my_static_text3 = wx.StaticText(self, label = "sender application", pos = (10, 120))
        self.my_static_text2 = wx.StaticText(self, label = "receiver buffer", pos = (500, 10))
        self.my_static_text4 = wx.StaticText(self, label = "receiver application", pos = (500, 120))

        self.my_static_text5 = wx.StaticText(self, label = "sender buffer", pos = (10, 10))
        self.my_static_text6 = wx.StaticText(self, label = "sender application", pos = (10, 120))
        self.my_static_text7 = wx.StaticText(self, label = "receiver buffer", pos = (500, 10))
        self.my_static_text8 = wx.StaticText(self, label = "receiver application", pos = (500, 120))

        self.my_static_text9 = wx.StaticText(self, label = "sender buffer", pos = (10, 10))
        self.my_static_text10 = wx.StaticText(self, label = "sender application", pos = (10, 120))
        self.my_static_text11 = wx.StaticText(self, label = "receiver buffer", pos = (500, 10))
        self.my_static_text12 = wx.StaticText(self, label = "receiver application", pos = (500, 120))

        
        self.my_gauge1 = wx.Gauge(self, range=self.buffer_size, size=(200, 20), pos = (50, 10))
        self.my_gauge1.SetValue(0)

        self.my_gauge2 = wx.Gauge(self, range=self.file_size, size=(200, 20), pos = (50, 60))
        self.my_gauge2.SetValue(0)

        self.my_gauge3 = wx.Gauge(self, range=self.buffer_size, size=(200, 20), pos = (580, 10))
        self.my_gauge3.SetValue(0)

        self.my_gauge4 = wx.Gauge(self, range=self.file_size, size=(200, 20), pos = (580, 60))
        self.my_gauge4.SetValue(0)
           
        self.my_gauge5 = wx.Gauge(self, range=self.buffer_size, size=(200, 20), pos = (50, 10))
        self.my_gauge5.SetValue(0)

        self.my_gauge6 = wx.Gauge(self, range=self.buffer_size, size=(200, 20), pos = (50, 10))
        self.my_gauge6.SetValue(0)

        self.my_gauge7 = wx.Gauge(self, range=self.buffer_size, size=(200, 20), pos = (50, 10))
        self.my_gauge7.SetValue(0)

        self.my_gauge8 = wx.Gauge(self, range=self.buffer_size, size=(200, 20), pos = (50, 10))
        self.my_gauge8.SetValue(0)

        self.my_gauge9 = wx.Gauge(self, range=self.buffer_size, size=(200, 20), pos = (50, 10))
        self.my_gauge9.SetValue(0)

        self.my_gauge10 = wx.Gauge(self, range=self.file_size, size=(200, 20), pos = (50, 10))
        self.my_gauge10.SetValue(0)

        self.my_gauge11 = wx.Gauge(self, range=self.buffer_size, size=(200, 20), pos = (50, 10))
        self.my_gauge11.SetValue(0)

        self.my_gauge12 = wx.Gauge(self, range=self.file_size, size=(200, 20), pos = (50, 10))
        self.my_gauge12.SetValue(0)

        self.hbox = wx.BoxSizer(wx.HORIZONTAL)
        self.vbox1 = wx.BoxSizer(wx.VERTICAL)
        self.vbox2 = wx.BoxSizer(wx.VERTICAL)
        self.vbox3 = wx.BoxSizer(wx.VERTICAL)
        self.vbox4 = wx.BoxSizer(wx.VERTICAL)

        self.vbox1.Add(self.my_static_text1, flag = wx.ALL, border = 4)
        self.vbox1.Add(self.my_static_text5, flag = wx.ALL, border = 4)
        self.vbox1.Add(self.my_static_text9, flag = wx.ALL, border = 4)
        self.vbox1.Add(self.my_static_text3, flag = wx.ALL, border = 4)
        self.vbox1.Add(self.my_static_text6, flag = wx.ALL, border = 4)
        self.vbox1.Add(self.my_static_text10, flag = wx.ALL, border = 4)
        #self.vbox1.Add(self.my_button, flag = wx.ALL, border = 4)

        self.vbox2.Add(self.my_gauge1, flag = wx.ALL, border = 4)
        self.vbox2.Add(self.my_gauge5, flag = wx.ALL, border = 4)
        self.vbox2.Add(self.my_gauge7, flag = wx.ALL, border = 4)
        self.vbox2.Add(self.my_gauge2, flag = wx.ALL, border = 4)
        self.vbox2.Add(self.my_gauge6, flag = wx.ALL, border = 4)
        self.vbox2.Add(self.my_gauge8, flag = wx.ALL, border = 4)

        
        self.vbox3.Add(self.my_static_text2, flag = wx.ALL, border = 4)
        self.vbox3.Add(self.my_static_text7, flag = wx.ALL, border = 4)
        self.vbox3.Add(self.my_static_text11, flag = wx.ALL, border = 4)

        self.vbox3.Add(self.my_static_text4, flag = wx.ALL, border = 4)
        self.vbox3.Add(self.my_static_text8, flag = wx.ALL, border = 4)
        self.vbox3.Add(self.my_static_text12, flag = wx.ALL, border = 4)
        
        self.vbox4.Add(self.my_gauge3, flag = wx.ALL, border = 4)
        self.vbox4.Add(self.my_gauge9, flag = wx.ALL, border = 4)
        self.vbox4.Add(self.my_gauge11, flag = wx.ALL, border = 4)
        self.vbox4.Add(self.my_gauge4, flag = wx.ALL, border = 4)
        self.vbox4.Add(self.my_gauge10, flag = wx.ALL, border = 4)
        
        self.vbox4.Add(self.my_gauge12, flag = wx.ALL, border = 4)


        self.hbox.Add(self.vbox1, flag = wx.ALL, border = 4)
        self.hbox.Add(self.vbox2, flag = wx.ALL, border = 4)
        self.hbox.Add(self.vbox3, flag = wx.ALL, border = 4)
        self.hbox.Add(self.vbox4, flag = wx.ALL, border = 4)

        self.SetSizer(self.hbox)
    def OnIdle(self, event):
        self.count1 = self.count1 +20
        if self.count1 >= self.buffer_size:
            self.count1 = self.buffer_size    
        self.my_gauge1.SetValue(self.count1)
        self.my_gauge2.SetValue(self.count1)
        if self.my_gauge2.GetValue() == self.buffer_size:
            self.count2 = self.count2 + 20
            if self.count2 >= self.buffer_size:
                self.count2 = self.buffer_size
            self.my_gauge3.SetValue(self.count2)
            self.count3 = self.count3 - 20 
            if self.count3 <= 8:
                self.count3 = 0
            self.my_gauge1.SetValue(self.count3)
            self.my_gauge2.SetValue(self.count3)
            
        if self.my_gauge1.GetValue() == 0 and self.my_gauge3.GetValue() == self.buffer_size:
            self.count4 = self.count4 + 20
            if self.count4 >= self.buffer_size:
                self.count4 = self.buffer_size
            self.my_gauge5.SetValue(self.count4)
            self.my_gauge3.SetValue(0)
            #self.my_gauge9.SetValue(2048)
        if self.my_gauge5.GetValue() == 2048:
            self.count5 = self.count5 + 20
            if self.count5 >= self.buffer_size:
                self.count5 = 0
            #self.my_gauge9.SetValue(0)
            self.my_gauge10.SetValue(self.file_block)
        if self.my_gauge10.GetValue() == 2048:
            self.my_gauge7.SetValue(2048)
            self.my_gauge5.SetValue(0)
            self.count6 = self.count6 - 20
            if self.count6 <= 8:
                self.count6 =0
            self.my_gauge7.SetValue(self.count6)
            self.my_gauge9.SetValue(self.buffer_size)
            self.count7 = self.count7 - 20
            if self.count7 <= 8:
                self.count7 = 0
                self.my_gauge9.SetValue(self.count7)
            self.my_gauge7.SetValue(2048)
            self.count9 = self.count9 - 20
            if self.count9 <= 8:
                self.count9 = 0
            self.my_gauge7.SetValue(self.count9)
            self.count10 = self.count10 + 20 
            if self.count10 >= 2048:
                self.count10 = 2048
            self.my_gauge11.SetValue(self.count10)
        if self.my_gauge10.GetValue() == 2048 and self.my_gauge11.GetValue() == 2048:
            self.my_gauge10.SetValue(0)
            self.my_gauge11.SetValue(0)
            self.my_gauge12.SetValue(self.file_size)

app = wx.App()
MyFrame(None, title = "flow control")
app.MainLoop()

## 3 Conclusion

By the project, I learn more about the principle of Go Back N and flow control. At the same time, My skill of programming has been enforced.